# Generating Qubit Hamiltonians

In [1]:
from utility import *
import numpy as np 

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(h4))

H:
-0.2434757373932887 [] +
-0.03422206211633393 [X0 X1 Y2 Y3] +
-0.034222062119673984 [X0 X1 Y4 Y5] +
-0.019399049458071443 [X0 X1 Y6 Y7] +
0.03422206211633393 [X0 Y1 Y2 X3] +
0.034222062119673984 [X0 Y1 Y4 X5] +
0.019399049458071443 [X0 Y1 Y6 X7] +
1.0071472006888387e-06 [X0 Z1 X2 X3 Z4 Z5 Z6 X7] +
1.0071472006888387e-06 [X0 Z1 X2 Y3 Z4 Z5 Z6 Y7] +
-0.014173745316695836 [X0 Z1 X2 X4 Z5 X6] +
-0.03361644836850833 [X0 Z1 X2 X5 Z6 X7] +
-0.03361644836850833 [X0 Z1 X2 Y5 Z6 Y7] +
-0.014173745316695843 [X0 Z1 Y2 Y4 Z5 X6] +
0.03361644836850833 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.01944270305181249 [X0 Z1 Z2 X3 X5 X6] +
-0.03361644836850833 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
0.01944270305181249 [X0 Z1 Z2 Y3 Y5 X6] +
-1.011601823068791e-06 [X0 Z1 Z2 Z3 X4 X5 Z6 X7] +
-1.011601823068791e-06 [X0 Z1 Z2 Z3 X4 Y5 Z6 Y7] +
-4.564020154629994e-07 [X0 Z1 Z2 Z3 Z4 Z5 X6] +
5.862191087644547e-07 [X0 Z1 Z2 Z3 Z4 X6] +
-4.2538271430433636e-07 [X0 Z1 Z2 Z3 Z5 X6] +
-5.81362230427443e-07 [X0 Z1 Z2 Z4 Z5 X6] +
4.257849

In [3]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h4)
h4_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in h4.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h4_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h4_matrix)
print("The ground state energy from S1: ")
obtain_PES('h4', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
converged SCF energy = -1.69488959079578
E = -1.91510654951181 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-1.91510655e+00 -1.90077950e+00 -1.90077950e+00 -1.90077950e+00
 -1.76431832e+00 -1.70868549e+00 -1.61197093e+00 -1.61197091e+00
 -1.61197070e+00 -1.61197067e+00 -1.58818682e+00 -1.58818680e+00
 -1.58818660e+00 -1.58818657e+00 -1.50408386e+00 -1.50408385e+00
 -1.50408383e+00 -1.50408375e+00 -1.50408372e+00 -1.50408371e+00
 -1.30442695e+00 -1.30442695e+00 -1.30442695e+00 -1.30442695e+00
 -1.19070391e+00 -1.19070366e+00 -1.18396031e+00 -1.18396031e+00
 -1.18070661e+00 -1.18070661e+00 -1.17945663e+00 -1.17945663e+00
 -1.17945660e+00 -1.17945651e+00 -1.17945648e+00 -1.17945648e+00
 -1.17543594e+00 -1.17543594e+00 -1.17543594e+00 -1.17543594e+00
 -1.16792324e+00 -1.16792324e+00 -1.14100921e+00 -1.14100921e+00
 -1.14100921e+00 -1.14100921e+00 -1.14100921e+00 -1.04423764e+00
 -1.04423764e+00 -1.02845060e+00 -1.02845060e+00 -9.72775

In [4]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h4, n_spin_orbitals=8, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.2434757373932887 [] +
-0.03361644836850833 [X0 X1 Z2 Z3 X4] +
-0.038997305666699005 [X0 Y1 Y2] +
0.03361644836850833 [X0 Y1 Y4] +
1.3904634951246442e-08 [X0 Z1 X2] +
0.03888540610362498 [X0 Z1 X2 X3 Z4] +
0.028347490633391686 [X0 Z1 X2 Z3 X4] +
-2.7809269909865458e-08 [X0 Z1 X2 Z4] +
0.03422206211633393 [X0 Z1 Z2] +
0.0369144841581601 [X0 Z1 Z2 X3 X4] +
-1.0071472006888387e-06 [X0 Z1 Z3 X4] +
-0.03422206211633393 [X0 Z1 Z4] +
-1.0044234829437865e-06 [X0 X2] +
1.0071472006888387e-06 [X0 Z2 X3 Z4] +
-0.03422206211633393 [X0 Z2 Z3] +
-1.0071472006888387e-06 [X0 X3] +
-0.0369144841581601 [X0 Y3 Y4] +
0.03422206211633393 [X0 Z3 Z4] +
1.0071472006888387e-06 [X0 X4] +
0.03361644836850833 [Y0 X1 Z2 Y3 Z4] +
-0.03361644836850833 [Y0 X1 Z2 Z3 Y4] +
-0.03361644836850833 [Y0 X1 Y3] +
0.038997305666699005 [Y0 Y1 X2] +
-0.03361644836850833 [Y0 Y1 Z2 X3 Z4] +
0.03361644836850833 [Y0 Y1 X3] +
-0.03361644836850833 [Y0 Y1 X4] +
1.3904634951246442e-08 [Y0 Z1 Y2] +
0.0388854

In [5]:
# Building the matrix representation of the effective Hamiltonian
h4_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h4_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]
